Radiomics features extraction  

E. Lavrova  
10.02.2020  


ds1 - CRC Liege (private): training, MS/non-MS subjects, T1w + qMRI (PD, MT, R1, R2*)   
ds2 - CC359 (https://sites.google.com/view/calgary-campinas-dataset/home/download): external validation, non-MS subjects, T1w  
ds3 - MSSEG (https://portal.fli-iam.irisa.fr/msseg-challenge/overview): external validation, MS subjects, T1w 
  
Don't forget to enable 'norm ROI' for T1w and disable for qMRI!

In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nilearn
import nibabel as nib
import glob
import ntpath
from tqdm import tqdm_notebook as tqdm
import sys
import SimpleITK as sitk
from radiomics import featureextractor
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# User-defined functions

Image preprocessing: N4 bias field correction in ROI, percentile filtering, intensity normalization in ROI

In [ ]:
def correctBiasField(img_input, mask):
    
    # N4 bias field correction (https://www.ncbi.nlm.nih.gov/pubmed/20378467) in ROI
    
    # input:
    # img_input - original image to be corrected (3D array)
    # mask - binary mask, representing ROI for correction (3D binary integer array with the same shape as img_input)
    
    # output:
    # corrected image (3D array)

    corrected = False
    img_output = np.zeros(img_input.shape)

    while not corrected:

        try:
            corrector = sitk.N4BiasFieldCorrectionImageFilter()
            inputImage = sitk.GetImageFromArray(img_input)
            maskImage = sitk.GetImageFromArray((mask > 0.9).astype(np.uint8))
            inputImage = sitk.Cast(inputImage, sitk.sitkFloat32)
            output = corrector.Execute(inputImage, maskImage)
            img_output = sitk.GetArrayFromImage(output)
            corrected = True
        except:
            print ('BFC failed')

    return img_output


def filterMedian(img_input, perc):
    
    # percentile filtering: 
    # threshold (non-linear) filtering with lower (perc percentile of image intensities distribution) 
    # and upper (100-perc percentile of image intensities distribution) thresholds;
    # all image intensities below lower threshold are to be re-initialized with lower threshold intensity,
    # all image intensities above upper threshold are to be re-initialized with upper threshold intensity
    
    # input:
    # img_input - original image to be corrected (3D array)
    # perc - percentile, defining lower threshold and symmetrical upper threshold (int < 50)
    # (2*perc = ratio of intensities to be re-initialized)
    
    # output:
    # filtered image (3D array)

    img_flattened = img_input.flatten()

    th_low = np.percentile(img_flattened, perc)
    th_up = np.percentile(img_flattened, 100 - perc)

    mask_low = img_input < th_low
    mask_up = img_input > th_up

    img_output = img_input * (1 - mask_up) + th_up * mask_up
    img_output = img_output * (1 - mask_low) + th_low * mask_low

    return img_output


def normROI(img_input, mask):
    
    # intensity normalization in ROI: Z-scoring
    
    # input:
    # img_input - original image to be corrected (3D array)
    # mask - binary mask, representing ROI for correction (3D binary integer array with the same shape as img_input)
    
    # output:
    # normalized image (3D array)
    
    mask_binary = (mask > 0.9).astype(int)
    mask_nan = mask_binary / mask_binary
    img_masked_nan = img_input * mask_nan

    m = np.nanmean(img_masked_nan)
    s = np.nanstd(img_masked_nan)

    img_output = (img_input - m) * mask_binary / s

    return img_output

Features extraction: from original image, from pre-cropped area, with fixed bin number (50), 7 feature classes (first order, shape, GLCM, GLRLM, GLSZM, GLDM, NGTDM)

In [ ]:
def extractFeatures(img_input, mask):
    
    # radiomics features extraction with pyradiomics package (https://www.radiomics.io)
    
    # input:
    # img_input - original image to be corrected (3D array)
    # mask - binary mask, representing ROI for correction (3D binary integer array with the same shape as img_input)
    
    # output:
    # f_v - feature vector (dictionary: f_v[feature] = value)

    i = sitk.GetImageFromArray(img_input)
    m = sitk.GetImageFromArray(mask)
    f_v = {}

    settings = {}
    settings['preCrop'] = True
    settings['normalize'] = False

    settings['binCount'] = 50
    settings['resampledPixelSpacing'] = None
    settings['interpolator'] = sitk.sitkBSpline

    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

    extractor.addProvenance(False)
    extractor.disableAllFeatures()
    extractor.enableImageTypes(Original={})

    extractor.enableFeatureClassByName('firstorder', enabled=True)
    extractor.enableFeatureClassByName('shape', enabled=True)
    extractor.enableFeatureClassByName('glcm', enabled=True)
    extractor.enableFeatureClassByName('glrlm', enabled=True)
    extractor.enableFeatureClassByName('glszm', enabled=True)
    extractor.enableFeatureClassByName('gldm', enabled=True)
    extractor.enableFeatureClassByName('ngtdm', enabled=True)

    extracted_ftrs = extractor.execute(i, m)

    for key, value in extracted_ftrs.items():
        f_v[key] = value

    return f_v


def createHeader():
    
    # creates dataset header for extractFeatures extractor
    
    # input:
    # void
    
    # output:
    # list: ['sub', '{feature_0}', '{feature_1}', ..., '{feature_N}']
    
    f = {'sub': ''}
    i_t = np.ones((2, 2, 2))
    m_t = np.zeros((2, 2, 2))
    m_t[0, 0, 0] = 1
    m_t[1, 1, 1] = 1
    f.update(extractFeatures(i_t, m_t))

    return list(f.keys())


def extractFrom3TissuesThroughSubFolders(ds_folder, ds_names, modality_name, 
                                        img_name_template, 
                                        roi_NAWM_name_template, 
                                        roi_GM_name_template, norm = True,
                                        roi_icv_name_template = 'icv', 
                                        roi_les_name_template = 'lesion', 
                                        roi_CSF_name_template = 'CSF'):
    
    # features extraction with extractFeatures function, performed through all the subjects of the given dataset
    # separate subjects' folders with imaging data (MRI + tissues masks) are to be situated in dataset directory
    # extracts features from the given image from 3 tissue types: WM, NAWM, GM
    # to form ICV mask for intensities normalization, either ICV mask, or CSF mask is required
    # if lesion mask is not presented, it is assumed, that roi_WM == roi_NAWM
    
    # input:
    # ds_folder: name of dataset directory (string)
    # ds_names: names of subjects from dataset to be included into the study (list of string)
    # modality_name: name of MRI modality to be indicated in output .csv file name (string)
    # img_name_template: MRI base filename pattern (string)
    # roi_NAWM_name_template: NAWM mask base filename pattern (string)
    # roi_GM_name_template: GM mask base filename pattern (string)
    # roi_icv_name_template: ICV mask base filename pattern (string)
    # roi_les_name_template: lesion mask base filename pattern (string)
    # roi_CSF_name_template: CSF mask base filename pattern (string)
    
    # output:
    # void: saves 'features_{dataset} _ {modality} _ {tissue}.csv' files with features values 
    # for different tissue types into dataset directory  
    
    ds_name = ds_folder.split('/')[-2]
    
    h = createHeader()

    csv_file_WM = open('../results/features/features_' + ds_name + '_' + modality_name + '_WM.csv', 'w')
    writer_WM = csv.DictWriter(csv_file_WM, fieldnames = h, lineterminator='\n')
    writer_WM.writeheader()

    csv_file_NAWM = open('../results/features/features_' + ds_name + '_' + modality_name + '_NAWM.csv', 'w')
    writer_NAWM = csv.DictWriter(csv_file_NAWM, fieldnames = h, lineterminator='\n')
    writer_NAWM.writeheader()

    csv_file_GM = open('../results/features/features_' + ds_name + '_' + modality_name + '_GM.csv', 'w')
    writer_GM = csv.DictWriter(csv_file_GM, fieldnames = h, lineterminator='\n')
    writer_GM.writeheader()
    
    
    with tqdm(total=len(ds_names), file=sys.stdout) as pbar:        
        for sub in ds_names:

            img_name = glob.glob(ds_folder + sub + '/' + img_name_template)[0]
            img_nii = nib.load(img_name)
            img = img_nii.get_fdata()
            
            roi_NAWM_name = glob.glob(ds_folder + sub + '/' + roi_NAWM_name_template)[0]
            roi_NAWM_nii = nib.load(roi_NAWM_name)
            roi_NAWM = roi_NAWM_nii.get_fdata()
            
            roi_WM = np.copy(roi_NAWM)

            roi_GM_name = glob.glob(ds_folder + sub + '/' + roi_GM_name_template)[0]
            roi_GM_nii = nib.load(roi_GM_name)
            roi_GM = roi_GM_nii.get_fdata()
            
            if len(glob.glob(ds_folder + sub + '/' + roi_les_name_template))>0:
                roi_les_name = glob.glob(ds_folder + sub + '/' + roi_les_name_template)[0]
                roi_les_nii = nib.load(roi_les_name)
                roi_WM += roi_les_nii.get_fdata()

            if len(glob.glob(ds_folder + sub + '/' + roi_icv_name_template))>0:
                roi_icv_name = glob.glob(ds_folder + sub + '/' + roi_icv_name_template)[0]
                roi_icv_nii = nib.load(roi_icv_name)
                roi_icv = roi_icv_nii.get_fdata()
                
            if len(glob.glob(ds_folder + sub + '/' + roi_CSF_name_template))>0:
                roi_CSF_name = glob.glob(ds_folder + sub + '/' + roi_CSF_name_template)[0]
                roi_CSF_nii = nib.load(roi_CSF_name)
                roi_icv = roi_CSF_nii.get_fdata() + roi_WM + roi_GM
                
            roi_icv = (roi_icv > 0.9).astype(int)
            roi_WM = (roi_WM > 0.9).astype(int)
            roi_NAWM = (roi_NAWM > 0.9).astype(int)
            roi_GM = (roi_GM > 0.9).astype(int)
            
            img_preproc = np.copy(img)

            if norm == True:
                img_preproc = normROI(img_preproc, roi_icv)

            f_sub_WM = {'sub': sub}
            f_sub_WM.update(extractFeatures(img_preproc, roi_WM))
            writer_WM.writerow(f_sub_WM)

            f_sub_NAWM = {'sub': sub}
            f_sub_NAWM.update(extractFeatures(img_preproc, roi_NAWM))
            writer_NAWM.writerow(f_sub_NAWM)

            f_sub_GM = {'sub': sub}
            f_sub_GM.update(extractFeatures(img_preproc, roi_GM))
            writer_GM.writerow(f_sub_GM)

            pbar.set_description('processed: %s' % sub)
            pbar.update(1)


    csv_file_WM.close()
    csv_file_NAWM.close()
    csv_file_GM.close()

In [2]:
ds1_folder = '../data/CRC/'
ds2_folder = '../data/CC359/'
ds3_folder = '../data/MSSEG/'

# Resampling

Images resampling to 1x1x1 voxel size (DS2 and DS3)

In [ ]:
# DS2
for i in range (1, 360):
    T1_filename = glob.glob(ds2_folder + '/CC0' + format(i, '03d') + '*.nii.gz')[0]
    T1_nii = nib.load(T1_filename)
    if T1_nii.header.get_zooms() != (1, 1, 1):
        T1_nii_resampled = nib.processing.resample_to_output(T1_nii, voxel_sizes=(1, 1, 1))
        nib.save(T1_nii_resampled, T1_filename)

In [ ]:
# DS3
for item in os.listdir(ds3_folder):
    path = os.path.join(subs_dir, item)
    if os.path.isdir(path): 
        T1_filename = path + '/3DT1.nii'
        T1_nii = nib.load(T1_filename)
        if T1_nii.header.get_zooms() != (1, 1, 1):     
            T1_nii_resampled = nib.processing.resample_to_output(T1_nii, voxel_sizes=(1, 1, 1))
            nib.save(T1_nii_resampled, T1_filename)

# Brain tissues segmentation

Performed in MATLAB with SPM12 (URL: http://www.fil.ion.ucl.ac.uk/spm).  
* DS1: for MSP - LST (URL: https://www.applied-statistics.de/lst.html) for lesions segmentation, then brain tissues segmentation with US-with-Lesion (URL: https://github.com/CyclotronResearchCentre/USwLesion) for hMRI (https://hmri-group.github.io/hMRI-toolbox/); for HCS - unified segmentation with hMRI;  
* DS2: unified segmentation with SPM12;  
* DS3: LST for lesions segmentation, then brain tissues segmentation with US-with-Lesion

## Mask correction for MSPA015

Cerebellum is not presented at the images -> need to crop the masks

In [ ]:
patterns = ['c1*.nii', 'c2*.nii', 'c3*.nii', 'icv*.nii']

for pattern in patterns:
    img_name = glob.glob(ds1_folder + '/MSPA015/' + pattern)[0]
    img_nii = nib.load(img_name)
    img = img_nii.get_fdata()
    img[:, :50, :] = np.zeros(img[:, :50, :].shape)
    img_nii = nib.Nifti1Image(img, img_nii.affine, img_nii.header)
    nib.save(img_nii, img_name)

# Bias field correction

DS1: BFC for T1w images in ICV; corrected image is to be saved in the same folder as the original one, under the same name, with '_preprocessed' index 

In [ ]:
for sub in ds1_names:
    
    t1w_name = glob.glob(ds1_folder + 'sub-' + sub + '/sub-*T1w*.nii')[0]
    t1w_nii = nib.load(t1w_name)
    t1w = t1w_nii.get_fdata()
    
    icv_name = glob.glob(ds1_folder + 'sub-' + sub + '/icv*.nii')[0]
    icv_nii = nib.load(icv_name)
    icv = icv_nii.get_fdata()
    
    t1w_bfc = correctBiasField(t1w, icv)
    t1w_bfc_nii = nib.Nifti1Image(t1w_bfc, t1w_nii.affine)
    nib.save(t1w_bfc_nii, ntpath.basename(t1w_name)[:-4] + '_preprocessed.nii')

DS2: BFC for T1w images in ICV (c1 + c2 + c3 masks == GM + WM + CSF masks); corrected image is to be saved in the same folder as the original one, under the same name, with '_preprocessed' index 

In [ ]:
for sub in ds2_names:
    
    t1w_name = glob.glob(ds2_folder + sub + '/' + sub + '*.nii')[0]
    t1w_nii = nib.load(t1w_name)
    t1w = t1w_nii.get_fdata()
    
    roi_GM_name = glob.glob(ds2_folder + sub + '/c1' + sub + '*.nii')[0]
    roi_GM_nii = nib.load(roi_GM_name)
    roi_GM = roi_GM_nii.get_fdata()
    
    roi_WM_name = glob.glob(ds2_folder + sub + '/c2' + sub + '*.nii')[0]
    roi_WM_nii = nib.load(roi_WM_name)
    roi_WM = roi_WM_nii.get_fdata()
    
    roi_CSF_name = glob.glob(ds2_folder + sub + '/c3' + sub + '*.nii')[0]
    roi_CSF_nii = nib.load(roi_CSF_name)
    roi_CSF = roi_CSF_nii.get_fdata()
    
    icv = ((roi_GM + roi_WM + roi_CSF) > 0.9).astype(int)
    
    t1w_bfc = correctBiasField(t1w, icv)
    t1w_bfc_nii = nib.Nifti1Image(t1w_bfc, t1w_nii.affine)
    nib.save(t1w_bfc_nii, ntpath.basename(t1w_name)[:-4] + '_preprocessed.nii')

DS3: BFC for T1w images in ICV; corrected image is to be saved in the same folder as the original one, under the same name, with '_preprocessed' index 

In [ ]:
for sub in ds3_names:
    
    t1w_name = ds3_folder + sub + '/3DT1.nii'
    t1w_nii = nib.load(t1w_name)
    t1w = t1w_nii.get_fdata()
    
    icv_name = ds3_folder + sub + '/icv_k3DT1.nii'
    icv_nii = nib.load(icv_name)
    icv = icv_nii.get_fdata()
    
    t1w_bfc = correctBiasField(t1w, icv)
    t1w_bfc_nii = nib.Nifti1Image(t1w_bfc, t1w_nii.affine)
    nib.save(t1w_bfc_nii, ds3_folder + sub + '/3DT1_preprocessed.nii')

# Subjects selection

Loading .csv files with demographic and acquisition description of participating subjects

In [3]:
ds1 = pd.read_csv('../data/description/CRC_description.csv')
ds2 = pd.read_csv('../data/description/CC359_description.csv')
ds3 = pd.read_csv('../data/description/MSSEG_description.csv')

ds1.set_index('sub', inplace = True)
ds2.set_index('sub', inplace = True)
ds3.set_index('name', inplace = True)

Subjects to remove from the datasets because of poor image quality

In [4]:
ds1_names_to_remove = []
ds2_names_to_remove = ['CC0087', 'CC0097', 'CC0209', 'CC0216', 'CC0220']
ds3_names_to_remove = []

ds1.drop(ds1_names_to_remove, axis = 0, inplace = True)
ds2.drop(ds2_names_to_remove, axis = 0, inplace = True)
ds3.drop(ds3_names_to_remove, axis = 0, inplace = True)

External validation subjects selection, based on:  
- age: should not be out of range, presented in training dataset ds1,  
- magnetic field strength: 3T, as training dataset ds1 contains 3T-scanner data only (note: in ds3, center with label '7', used 1.5-scanner data)

In [5]:
age_min = ds1.age.min()
age_max = ds1.age.max()

ds2_selected = ds2.loc[(ds2.field == 3)&(ds2.age <= age_max)&(ds2.age >= age_min)]
ds3_selected = ds3.loc[(ds3.center != 7)&(ds3.age <= age_max)&(ds3.age >= age_min)]

Data description for ds1, ds2, ds3 after validation subjects selection

In [6]:
description = []
description.append({'ds': 'ds1', 'size': len(ds1),
                    'age_mean': ds1.age.mean(), 'age_std': ds1.age.std(),
                    'age_min': ds1.age.min(), 'age_max': ds1.age.max(),
                    'gender_male': len(ds1.loc[ds1.gender == 'M']), 
                    'gender_female': len(ds1.loc[ds1.gender == 'F'])})
description.append({'ds': 'ds2', 'size': len(ds2_selected),
                    'age_mean': ds2_selected.age.mean(), 'age_std': ds2_selected.age.std(),
                    'age_min': ds2_selected.age.min(), 'age_max': ds2_selected.age.max(),  
                    'gender_male': len(ds2_selected.loc[ds2_selected.gender == 'M']), 
                    'gender_female': len(ds2_selected.loc[ds2_selected.gender == 'F'])})
description.append({'ds': 'ds3', 'size': len(ds3_selected),
                    'age_mean': ds3_selected.age.mean(), 'age_min': ds3_selected.age.min(), 
                    'age_max': ds3_selected.age.max(), 'age_std': ds3_selected.age.std(), 
                    'gender_male': len(ds3_selected.loc[ds3_selected.gender == 'M']), 
                    'gender_female': len(ds3_selected.loc[ds3_selected.gender == 'F'])})
description = pd.DataFrame(description, 
                           columns = ['ds', 'size', 
                                      'age_mean', 'age_std', 'age_min', 'age_max', 
                                      'gender_male', 'gender_female'])

description

,ds,size,age_mean,age_std,age_min,age_max,gender_male,gender_female
0,ds1,72,45.791667,12.056133,21,65,31,41
1,ds2,167,52.706587,7.309271,29,65,82,85
2,ds3,10,40.500000,10.772908,24,55,5,5


Comparison of age and gender variable distributions between HCS and MSP in DS1 and validation data, and between development and validation data

In [ ]:
# age in HCS and MSP cohorts of DS1 (Mann-Whitney test)
sp.stats.mannwhitneyu(ds1.loc[ds1.outcome == 0].age, ds1.loc[ds1.outcome == 1].age, alternative= 'two-sided')

In [ ]:
# gender in HCS and MSP cohorts of DS2 (Fisher's exact test)
sp.stats.fisher_exact([[len(ds1.loc[ds1.outcome == 0].loc[ds1.loc[ds1.outcome == 0].gender == 'M']), 
                        len(ds1.loc[ds1.outcome == 1].loc[ds1.loc[ds1.outcome == 1].gender == 'M'])], 
                       [len(ds1.loc[ds1.outcome == 0].loc[ds1.loc[ds1.outcome == 0].gender == 'F']), 
                        len(ds1.loc[ds1.outcome == 1].loc[ds1.loc[ds1.outcome == 1].gender == 'F'])]])

In [ ]:
# age in HCS and MSP cohorts of validation data (DS2+DS3) (Mann-Whitney test)
sp.stats.mannwhitneyu(ds2_selected.age, ds3_selected.age, alternative= 'two-sided')

In [ ]:
# gender in HCS and MSP cohorts of validation data (DS2+DS3) (Fisher's exact test)
sp.stats.fisher_exact([[len(ds3_selected.loc[ds3_selected.gender == 'M']), 
                        len(ds2_selected.loc[ds2_selected.gender == 'M'])], 
                       [len(ds3_selected.loc[ds3_selected.gender == 'F']), 
                        len(ds2_selected.loc[ds2_selected.gender == 'F'])]])

In [ ]:
# age in development (DS1) and validation (DS2+DS3) data (Mann-Whitney test)
sp.stats.mannwhitneyu(ds1.age, pd.concat([ds1.age, ds3_selected.age]), alternative= 'two-sided')

In [ ]:
# gender in development (DS1) and validation (DS2+DS3) data (Fisher's exact test)
sp.stats.fisher_exact([[len(ds3_selected.loc[ds3_selected.gender == 'M'])+len(ds2_selected.loc[ds2_selected.gender == 'M']), 
                        len(ds1.loc[ds1.gender == 'M'])], 
                       [len(ds3_selected.loc[ds3_selected.gender == 'F'])+len(ds2_selected.loc[ds2_selected.gender == 'F']), 
                        len(ds1.loc[ds1.gender == 'F'])]])

Lists of subjects from ds1, ds2, ds3, included in analysis: create and save to txt files

In [ ]:
ds1_names = list(ds1.index)
ds2_names = list(ds2_selected.index)
ds3_names = list(ds3_selected.index)

with open("../results/names/ds1_names.txt", "w") as output:
    for name in ds1_names:
        output.write(str(name) + '\n')

with open("../results/names/ds2_names.txt", "w") as output:
    for name in ds2_names:
        output.write(str(name) + '\n')
        
with open("../results/names/ds3_names.txt", "w") as output:
    for name in ds3_names:
        output.write(str(name) + '\n')

# Features extraction

## Conventional MRI: T1w

ds1: extraction of T1w-based features from WM, NAWM, GM; feature values are sent to .csv file 'features_{dataset} _ {modality} _ {tissue}.csv'

In [ ]:
extractFrom3TissuesThroughSubFolders(ds1_folder, ds1_names, 'T1w',
                                    'T1_preprocessed.nii', 
                                    'c2ktsub*.nii', 
                                    'c1ktsub*.nii', 
                                    roi_icv_name_template = 'icv*.nii', 
                                    roi_les_name_template = 'c3ktsub*.nii')

ds2: extraction of T1w-based features from WM, NAWM, GM; feature values are sent to .csv file 'features_{dataset} _ {modality} _ {tissue}.csv'

In [ ]:
extractFrom3TissuesThroughSubFolders(ds2_folder, ds2_names, 'T1w',
                                    'CC*.nii', 
                                    'c2CC*.nii', 
                                    'c1CC*.nii',  
                                    roi_CSF_name_template = 'c3CC*.nii')

ds3: extraction of T1w-based features from WM, NAWM, GM; feature values are sent to .csv file 'features_{dataset} _ {modality} _ {tissue}.csv'

In [ ]:
extractFrom3TissuesThroughSubFolders(ds3_folder, ds3_names, 'T1w',
                                    '3DT1.nii', 
                                    'c2k3DT1.nii', 
                                    'c1k3DT1.nii',
                                     roi_icv_name_template = 'icv_k3DT1.nii',
                                     roi_les_name_template = 'c3k3DT1.nii')

## qMRI

Extraction of qMRI-based features from WM, NAWM, GM; feature values are sent to .csv file 'features_{dataset} _ {modality} _ {tissue}.csv';  
For ds1 only, as it uniquely contains qMRI data

PD

In [ ]:
extractFrom3TissuesThroughSubFolders(ds1_folder, ds1_names, 'PD',
                                    'ktsub*_PD.nii', 
                                    'c2ktsub*.nii', 
                                    'c1ktsub*.nii', norm = False,
                                    roi_icv_name_template = 'icv*.nii', 
                                    roi_les_name_template = 'c3ktsub*.nii')

MT

In [ ]:
extractFrom3TissuesThroughSubFolders(ds1_folder, ds1_names, 'MT',
                                    'ktsub*_MT.nii', 
                                    'c2ktsub*.nii', 
                                    'c1ktsub*.nii', norm = False,
                                    roi_icv_name_template = 'icv*.nii', 
                                    roi_les_name_template = 'c3ktsub*.nii')

R1

In [ ]:
extractFrom3TissuesThroughSubFolders(ds1_folder, ds1_names, 'R1',
                                    'ktsub*_R1.nii', 
                                    'c2ktsub*.nii', 
                                    'c1ktsub*.nii', norm = False,
                                    roi_icv_name_template = 'icv*.nii', 
                                    roi_les_name_template = 'c3ktsub*.nii')

R2*

In [ ]:
extractFrom3TissuesThroughSubFolders(ds1_folder, ds1_names, 'R2s',
                                    'ktsub*_R2s_OLS.nii', 
                                    'c2ktsub*.nii', 
                                    'c1ktsub*.nii', norm = False,
                                    roi_icv_name_template = 'icv*.nii', 
                                    roi_les_name_template = 'c3ktsub*.nii')

Merging datasets to get combined qMRI data

WM

In [ ]:
filename_PD = "../results/features/features_CRC_PD_WM.csv"
filename_MT = "../results/features/features_CRC_MT_WM.csv"
filename_R1 = "../results/features/features_CRC_R1_WM.csv"
filename_R2 = "../results/features/features_CRC_R2s_WM.csv"

df_PD = pd.read_csv(filename_PD)
df_MT = pd.read_csv(filename_MT)
df_R1 = pd.read_csv(filename_R1)
df_R2 = pd.read_csv(filename_R2)

df_PD.set_index('sub', inplace = True)
df_MT.set_index('sub', inplace = True)
df_R1.set_index('sub', inplace = True)
df_R2.set_index('sub', inplace = True)

features = df_PD.columns[14:-1]

for feature in features:
    
    df_PD.rename(columns={feature: 'PD_' + feature[9:]}, inplace=True)
    df_MT.rename(columns={feature: 'MT_' + feature[9:]}, inplace=True)
    df_R1.rename(columns={feature: 'R1_' + feature[9:]}, inplace=True)
    df_R2.rename(columns={feature: 'R2_' + feature[9:]}, inplace=True)
    
df = pd.concat([df_PD[df_PD.columns[:-1]], df_MT[df_MT.columns[14:-1]], 
                df_R1[df_R1.columns[14:-1]], 
                df_R2[df_R2.columns[14:-1]], df_MT.outcome], axis = 1)

df.to_csv("../results/features/features_CRC_qMRI_WM.csv")

NAWM

In [ ]:
filename_PD = "../results/features/features_CRC_PD_NAWM.csv"
filename_MT = "../results/features/features_CRC_MT_NAWM.csv"
filename_R1 = "../results/features/features_CRC_R1_NAWM.csv"
filename_R2 = "../results/features/features_CRC_R2s_NAWM.csv"

df_PD = pd.read_csv(filename_PD)
df_MT = pd.read_csv(filename_MT)
df_R1 = pd.read_csv(filename_R1)
df_R2 = pd.read_csv(filename_R2)

df_PD.set_index('sub', inplace = True)
df_MT.set_index('sub', inplace = True)
df_R1.set_index('sub', inplace = True)
df_R2.set_index('sub', inplace = True)

features = df_PD.columns[14:-1]

for feature in features:
    
    df_PD.rename(columns={feature: 'PD_' + feature[9:]}, inplace=True)
    df_MT.rename(columns={feature: 'MT_' + feature[9:]}, inplace=True)
    df_R1.rename(columns={feature: 'R1_' + feature[9:]}, inplace=True)
    df_R2.rename(columns={feature: 'R2_' + feature[9:]}, inplace=True)
    
df = pd.concat([df_PD[df_PD.columns[:-1]], df_MT[df_MT.columns[14:-1]], 
                df_R1[df_R1.columns[14:-1]], 
                df_R2[df_R2.columns[14:-1]], df_MT.outcome], axis = 1)

df.to_csv("../results/features/features_CRC_qMRI_NAWM.csv")

GM

In [ ]:
filename_PD = "../results/features/features_CRC_PD_GM.csv"
filename_MT = "../results/features/features_CRC_MT_GM.csv"
filename_R1 = "../results/features/features_CRC_R1_GM.csv"
filename_R2 = "../results/features/features_CRC_R2s_GM.csv"

df_PD = pd.read_csv(filename_PD)
df_MT = pd.read_csv(filename_MT)
df_R1 = pd.read_csv(filename_R1)
df_R2 = pd.read_csv(filename_R2)

df_PD.set_index('sub', inplace = True)
df_MT.set_index('sub', inplace = True)
df_R1.set_index('sub', inplace = True)
df_R2.set_index('sub', inplace = True)

features = df_PD.columns[14:-1]

for feature in features:
    
    df_PD.rename(columns={feature: 'PD_' + feature[9:]}, inplace=True)
    df_MT.rename(columns={feature: 'MT_' + feature[9:]}, inplace=True)
    df_R1.rename(columns={feature: 'R1_' + feature[9:]}, inplace=True)
    df_R2.rename(columns={feature: 'R2_' + feature[9:]}, inplace=True)
    
df = pd.concat([df_PD[df_PD.columns[:-1]], df_MT[df_MT.columns[14:-1]], 
                df_R1[df_R1.columns[14:-1]], 
                df_R2[df_R2.columns[14:-1]], df_MT.outcome], axis = 1)

df.to_csv("../results/features/features_CRC_qMRI_GM.csv")

Adding outcome column: 0 - HCS, 1 - MSPA

In [ ]:
# DS1

ds1_outcomes = np.concatenate((np.zeros(36), np.ones(36))).astype(int)
roi_list = ['WM', 'NAWM', 'GM']
mod_list = ['T1w', 'PD', 'MT', 'R1', 'R2s']

filenames = glob.glob('../results/features/features_CRC*M.csv')
for file in filenames:
    df = pd.read_csv(filenames)
    df['outcome'] = ds1_outcomes
    df.to_csv(filename)

In [ ]:
# DS2

filenames = glob.glob('../results/features/features_CC359*.csv')
for file in filenames:
    df = pd.read_csv(filenames)
    df['outcome'] = np.zeros(len(df))
    df.to_csv(filename)

In [ ]:
# DS3

filenames = glob.glob('../results/features/features_MSSEG*.csv')
for file in filenames:
    df = pd.read_csv(filenames)
    df['outcome'] = np.ones(len(df))
    df.to_csv(filename)

# Features analysis

In [8]:
from scipy import stats
from sklearn.metrics import auc
import scipy as sp
from statsmodels.stats import multitest

In [9]:
def computeUnivarAUC(f_vector, outcome):
    
    # estimation of AUC for binary classifier, built on input feature
    
    # input:
    # f_vector - values of the given feature (1D array of float),
    # outcome - values of binary outcome, corresponding to observations in f_vector (1D array of integer)
    
    # output:
    # auc - AUC value (float)

    mean_0 = np.mean(f_vector[np.where(outcome == 0)])
    mean_1 = np.mean(f_vector[np.where(outcome == 1)])

    feature_min = np.min(f_vector)
    feature_max = np.max(f_vector)

    step = (feature_max - feature_min)/100

    P = np.sum(outcome == 1)
    N = np.sum(outcome == 0)

    th = []
    tprs = []
    fprs = []
    
    if mean_0 > mean_1:

        for i in range (0, 102):
            t = feature_min + i*step
            th.append(t)
            y_pred = (f_vector <= t).astype(int)
            tp = np.sum(y_pred[np.where(outcome == 1)])
            fp = np.sum(y_pred[np.where(outcome == 0)])
            tpr = tp/P
            fpr = fp/N
            tprs.append(tpr)
            fprs.append(fpr)
            
    else:
        
        for i in range (0, 102):
            t = feature_min + i*step
            th.append(t)
            y_pred = (f_vector > t).astype(int)
            tp = np.sum(y_pred[np.where(outcome == 1)])
            fp = np.sum(y_pred[np.where(outcome == 0)])
            tpr = tp/P
            fpr = fp/N
            tprs.append(tpr)
            fprs.append(fpr)
            
    return (auc(fprs, tprs))

reading features datasets

In [11]:
f = open('../results/names/ds1_names.txt', 'r')
ds_names = f.read().split('\n')
f.close()
ds_names = ds_names[:-1]

ds_names_HC = [s for s in ds_names if 'MSHS' in s]
ds_names_PA = [s for s in ds_names if 'MSPA' in s]

filename_descr = '../data/description/CRC_description.csv'
filename_featr = ['../results/features/features_CRC_T1w_WM.csv', '../results/features/features_CRC_PD_WM.csv', 
                  '../results/features/features_CRC_MT_WM.csv', '../results/features/features_CRC_R1_WM.csv', 
                  '../results/features/features_CRC_R2s_WM.csv', '../results/features/features_CRC_T1w_NAWM.csv', 
                  '../results/features/features_CRC_PD_NAWM.csv', '../results/features/features_CRC_MT_NAWM.csv', 
                  '../results/features/features_CRC_R1_NAWM.csv', '../results/features/features_CRC_R2s_NAWM.csv', 
                  '../results/features/features_CRC_T1w_GM.csv', '../results/features/features_CRC_PD_GM.csv', 
                  '../results/features/features_CRC_MT_GM.csv', '../results/features/features_CRC_R1_GM.csv', 
                  '../results/features/features_CRC_R2s_GM.csv']

ds_descr = pd.read_csv(filename_descr, index_col=0)
ds_descr = ds_descr.reindex(ds_names)

ds_featr = {}
modrois = []

for filename in filename_featr:
    modroi = filename.split('_')[-2] + '_' + filename.split('_')[-1][:-4]
    modrois.append(modroi)
    ds = pd.read_csv(filename, index_col=0)
    ds = ds.reindex(ds_names)
    ds_featr[modroi] = ds

features = ds_featr[modrois[0]].columns[14:-1]

for feature in features:
    
    for modroi in modrois:
        ds_featr[modroi].rename(columns={feature: modroi + '_' + feature[9:]}, inplace=True)

df = ds_featr[modrois[0]]

for modroi in modrois[1:]:
    df = pd.concat([df, ds_featr[modroi][ds_featr[modroi].columns[14:-1]]], axis = 1)
    
features = list(df.columns.values)
features.remove('outcome')

calculating characteristics for every feature

In [12]:
feature_characteristics = []
p_MW = []

for feature in features:
    rec = {'feature': feature}
    y = np.array(df[feature])
    rec['age_corr'] = stats.spearmanr(ds_descr.age, y)[0]
    rec['volume_corr'] = stats.spearmanr(df['original_shape_MeshVolume'], y)[0]
    rec['outcome_corr'] = stats.spearmanr(df['outcome'], y)[0]
    rec['univar_auc'] = computeUnivarAUC(y, df['outcome'])
    feature_characteristics.append(rec)
    s, p = sp.stats.mannwhitneyu(df.reindex(ds_names_HC)[feature].astype(float), 
                                 df.reindex(ds_names_PA)[feature].astype(float), 
                                 alternative = 'two-sided')
    p_MW.append(p)
    
reject, p_MW_corr, a_S_corr, a_B_corr = multitest.multipletests(p_MW, alpha = 0.01, method = 'bonferroni')

feature_characteristics = pd.DataFrame(feature_characteristics)
feature_characteristics.set_index('feature', inplace = True)
feature_characteristics['p_MW_corr'] = p_MW_corr

saving to .csv

In [ ]:
feature_characteristics.to_csv('../results/features_analysis/features_analysis.csv')

In [ ]:
feature_characteristics = pd.read_csv('../results/features_analysis/features_analysis.csv', index_col = 0)

Checking the features, which are correlated to age, volume, and outcome (threshold r = 0.85), and have potential to distinguish between HCS and MSP in terms of univariate AUC (threshold AUC = 0.75), and Mann-Whitney test for means (threshold p = 0.01, with Bonferroni correction) 

In [13]:
# setting the threshold
age_corr_th = np.abs(feature_characteristics.age_corr) > 0.85
volume_corr_th = np.abs(feature_characteristics.volume_corr) > 0.85
outcome_corr_th = np.abs(feature_characteristics.outcome_corr) > 0.85
univar_auc_th = np.abs(feature_characteristics.univar_auc) > 0.75
p_MW_th = np.abs(feature_characteristics.p_MW_corr) < 0.01

In [14]:
# getting the features
age_corr_pos = age_corr_th.loc[age_corr_th == True].index.values
volume_corr_pos = volume_corr_th.loc[volume_corr_th == True].index.values
outcome_corr_pos = outcome_corr_th.loc[outcome_corr_th == True].index.values
univar_auc_pos = univar_auc_th.loc[univar_auc_th == True].index.values
p_MW_pos = p_MW_th.loc[p_MW_th == True].index.values

In [15]:
# counting detected features within each ROI and image type
age_corr_pos_df = []
age_corr_pos_n_df = []
volume_corr_pos_df = []
volume_corr_pos_n_df = []
outcome_corr_pos_df = []
outcome_corr_pos_n_df = []
univar_auc_pos_df = []
univar_auc_pos_n_df = []
p_MW_pos_df = []
p_MW_pos_n_df = []

roi_list = ['WM', 'NAWM', 'GM']
mod_list = ['T1w', 'PD', 'MT', 'R1', 'R2s']
for roi in roi_list:
    
    age_corr_rec = {'ROI': roi}
    age_corr_n_rec = {'ROI': roi}
    volume_corr_rec = {'ROI': roi}
    volume_corr_n_rec = {'ROI': roi}
    outcome_corr_rec = {'ROI': roi}
    outcome_corr_n_rec = {'ROI': roi}
    univar_auc_rec = {'ROI': roi}
    univar_auc_n_rec = {'ROI': roi}
    p_MW_rec = {'ROI': roi}
    p_MW_n_rec = {'ROI': roi}
    
    for mod in mod_list:    
        age_corr_rec[mod] = [s for s in age_corr_pos if (mod + '_' + roi) in s]
        age_corr_n_rec[mod] = len([s for s in age_corr_pos if (mod + '_' + roi) in s])
        volume_corr_rec[mod] = [s for s in volume_corr_pos if (mod + '_' + roi) in s]
        volume_corr_n_rec[mod] = len([s for s in volume_corr_pos if (mod + '_' + roi) in s])
        outcome_corr_rec[mod] = [s for s in outcome_corr_pos if (mod + '_' + roi) in s]
        outcome_corr_n_rec[mod] = len([s for s in outcome_corr_pos if (mod + '_' + roi) in s])
        univar_auc_rec[mod] = [s for s in univar_auc_pos if (mod + '_' + roi) in s]
        univar_auc_n_rec[mod] = len([s for s in univar_auc_pos if (mod + '_' + roi) in s])
        p_MW_rec[mod] = [s for s in p_MW_pos if (mod + '_' + roi) in s]
        p_MW_n_rec[mod] = len([s for s in p_MW_pos if (mod + '_' + roi) in s])
        
    age_corr_pos_df.append(age_corr_rec)
    age_corr_pos_n_df.append(age_corr_n_rec)
    volume_corr_pos_df.append(volume_corr_rec)
    volume_corr_pos_n_df.append(volume_corr_n_rec)
    outcome_corr_pos_df.append(outcome_corr_rec)
    outcome_corr_pos_n_df.append(outcome_corr_n_rec)
    univar_auc_pos_df.append(univar_auc_rec)
    univar_auc_pos_n_df.append(univar_auc_n_rec)
    p_MW_pos_df.append(p_MW_rec)
    p_MW_pos_n_df.append(p_MW_n_rec)
    
age_corr_pos_df = pd.DataFrame(age_corr_pos_df)
age_corr_pos_n_df = pd.DataFrame(age_corr_pos_n_df)
volume_corr_pos_df = pd.DataFrame(volume_corr_pos_df)
volume_corr_pos_n_df = pd.DataFrame(volume_corr_pos_n_df)
outcome_corr_pos_df = pd.DataFrame(outcome_corr_pos_df)
outcome_corr_pos_n_df = pd.DataFrame(outcome_corr_pos_n_df)
univar_auc_pos_df = pd.DataFrame(univar_auc_pos_df)
univar_auc_pos_n_df = pd.DataFrame(univar_auc_pos_n_df)
p_MW_pos_df = pd.DataFrame(p_MW_pos_df)
p_MW_pos_n_df = pd.DataFrame(p_MW_pos_n_df)

age_corr_pos_df.set_index('ROI', inplace = True)
age_corr_pos_n_df.set_index('ROI', inplace = True)
volume_corr_pos_df.set_index('ROI', inplace = True)
volume_corr_pos_n_df.set_index('ROI', inplace = True)
outcome_corr_pos_df.set_index('ROI', inplace = True)
outcome_corr_pos_n_df.set_index('ROI', inplace = True)
univar_auc_pos_df.set_index('ROI', inplace = True)
univar_auc_pos_n_df.set_index('ROI', inplace = True)
p_MW_pos_df.set_index('ROI', inplace = True)
p_MW_pos_n_df.set_index('ROI', inplace = True)

In [16]:
# distribution of the detected features in ROI and image types
print ('Number of features with age correlation')
display (age_corr_pos_n_df)

print ('Number of features with volume correlation')
display (volume_corr_pos_n_df)

print ('Number of features with outcome correlation')
display (outcome_corr_pos_n_df)

print ('Number of features with high univariate AUC')
display (univar_auc_pos_n_df)

print ('Number of features with statistically significant difference in groups')
display (p_MW_pos_n_df)

Number of features with age correlation


,T1w,PD,MT,R1,R2s
ROI,,,,,
WM,0,0,0,0,0
NAWM,0,0,0,0,0
GM,0,0,0,0,0


Number of features with volume correlation


,T1w,PD,MT,R1,R2s
ROI,,,,,
WM,0,3,1,1,0
NAWM,0,3,1,1,0
GM,0,0,0,0,0


Number of features with outcome correlation


,T1w,PD,MT,R1,R2s
ROI,,,,,
WM,0,0,0,1,0
NAWM,0,0,0,0,0
GM,0,0,0,0,0


Number of features with high univariate AUC


,T1w,PD,MT,R1,R2s
ROI,,,,,
WM,13,62,21,45,52
NAWM,8,28,57,9,37
GM,3,7,26,5,22


Number of features with statistically significant difference in groups


,T1w,PD,MT,R1,R2s
ROI,,,,,
WM,9,41,10,37,7
NAWM,0,12,42,5,26
GM,1,0,18,2,10
